In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración de gráficos
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

# --- 1. CARGA DE DATOS ---
file_path = 'datos_recolector.csv' 
output_path = 'datos_limpios.csv'

try:
    df = pd.read_csv(file_path)
    print(f"1. Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
except FileNotFoundError:
    print("Error: No se encuentra el archivo.")
    raise

cols_sensores = [c for c in df.columns if c.startswith('S_')]
cols_acciones = [c for c in df.columns if c.startswith('A_')]

X = df[cols_sensores]
y = df[cols_acciones]

# --- 2. LIMPIEZA AUTOMÁTICA (Columnas Constantes) ---
std_dev = X.std()
cols_constantes = std_dev[std_dev == 0].index.tolist()
if cols_constantes:
    print(f"\n2. Eliminando {len(cols_constantes)} columnas de sensores constantes.")
    X = X.drop(columns=cols_constantes)

# --- 2.5 LIMPIEZA MANUAL (Tu petición) ---
sensores_basura = ['S_damage', 'S_distFromStart', 'S_lastLapTime', 'S_distRaced', 'S_fuel']
a_borrar_manual = [c for c in sensores_basura if c in X.columns]

if a_borrar_manual:
    print(f"\n2.5 Eliminando sensores manuales: {a_borrar_manual}")
    X = X.drop(columns=a_borrar_manual)

# --- 3. LIMPIEZA AUTOMÁTICA (Columnas Redundantes) ---
UMBRAL_CORRELACION = 0.95 
print(f"\n3. Analizando correlaciones entre columnas (Umbral: {UMBRAL_CORRELACION*100}%)...")

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > UMBRAL_CORRELACION)]

if to_drop:
    print(f"   Eliminando {len(to_drop)} sensores redundantes.")
    X = X.drop(columns=to_drop)
else:
    print("   No se encontraron sensores redundantes.")

# --- 4. UNIFICACIÓN Y LIMPIEZA DE FILAS (NUEVO) ---
print("-" * 60)
print("4. Procesando filas...")

# Unimos todo en el dataframe final
df_final = pd.concat([X, y], axis=1)

# Contamos cuántas hay antes de limpiar
filas_antes = df_final.shape[0]

# --- AQUÍ ELIMINAMOS LAS FILAS DUPLICADAS ---
df_final = df_final.drop_duplicates()

filas_despues = df_final.shape[0]
filas_borradas = filas_antes - filas_despues

if filas_borradas > 0:
    print(f"   [!] Se eliminaron {filas_borradas} FILAS duplicadas.")
else:
    print("   No se encontraron filas duplicadas.")

# --- 5. GUARDAR ---
df_final.to_csv(output_path, index=False)

print("-" * 60)
print(f"¡ÉXITO! Archivo '{output_path}' generado.")
print(f"Dimensiones finales: {df_final.shape[0]} filas x {df_final.shape[1]} columnas")
print("Columnas finales de sensores:")
print(list(X.columns))

1. Datos cargados: 125636 filas, 38 columnas

2. Eliminando 5 columnas de sensores constantes.

3. Analizando correlaciones entre columnas (Umbral: 95.0%)...
   Eliminando 6 sensores redundantes.
------------------------------------------------------------
4. Procesando filas...
   [!] Se eliminaron 104 FILAS duplicadas.
------------------------------------------------------------
¡ÉXITO! Archivo 'datos_limpios.csv' generado.
Dimensiones finales: 125532 filas x 27 columnas
Columnas finales de sensores:
['S_angle', 'S_gear', 'S_rpm', 'S_speed', 'S_track_0', 'S_track_3', 'S_track_4', 'S_track_5', 'S_track_6', 'S_track_7', 'S_track_8', 'S_track_9', 'S_track_10', 'S_track_11', 'S_track_12', 'S_track_13', 'S_track_14', 'S_track_15', 'S_track_16', 'S_track_17', 'S_trackPos', 'S_wheelSpinVel_0', 'S_z']


In [2]:
# ==============================================================================
# AUDITORÍA DEFINITIVA: EL "TRIPLE JUEZ" (CORRELACIÓN + MI + RANDOM FOREST)
# ==============================================================================

# Definición de colores ANSI
C_RED = '\033[91m'
C_YELLOW = '\033[93m'
C_GREEN = '\033[92m'
C_RESET = '\033[0m'

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# 0. SINCRONIZACIÓN DE SEGURIDAD (Para evitar el error de longitud)
# ------------------------------------------------------------------------------
if isinstance(X, np.ndarray):
    X = pd.DataFrame(X, columns=[f'S_{i}' for i in range(X.shape[1])])
if isinstance(y, np.ndarray):
    nombres_y = ['A_Accel',"A_Brake", 'A_Gear', 'A_Steer']
    if y.shape[1] == 4:
        y = pd.DataFrame(y, columns=nombres_y)
    else:
        y = pd.DataFrame(y, columns=[f'Action_{i}' for i in range(y.shape[1])])

common_indices = X.index.intersection(y.index)
X = X.loc[common_indices]
y = y.loc[common_indices]
print(f"✅ Datos sincronizados: {len(X)} filas analizadas.")

# ------------------------------------------------------------------------------
# BUCLE DE ANÁLISIS POR ACCIÓN
# ------------------------------------------------------------------------------
acciones = y.columns.tolist() 

for accion in acciones:
    print(f"\n" + "="*60)
    print(f" 🔍 ANALIZANDO: {accion.upper()}")
    print("="*60)
    
    # --- JUEZ 1: CORRELACIÓN LINEAL (PEARSON) ---
    print("   1. Calculando Correlación Lineal...")
    # Calculamos solo la columna que nos interesa para ahorrar memoria
    # Usamos .abs() porque nos importa la fuerza, no si es positiva o negativa
    corr_scores = X.apply(lambda col: col.corr(y[accion])).abs().fillna(0)
    
    # --- JUEZ 2: MUTUAL INFORMATION (NO LINEAL) ---
    print("   2. Calculando Información Mutua (Complejidad)...")
    mi_raw = mutual_info_regression(X, y[accion], random_state=42)
    mi_scores = pd.Series(mi_raw, index=X.columns)
    
    # --- JUEZ 3: RANDOM FOREST (IMPORTANCIA REAL) ---
    print("   3. Entrenando Random Forest (Prueba de fuego)...")
    rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    rf.fit(X, y[accion])
    rf_scores = pd.Series(rf.feature_importances_, index=X.columns)
    
    # --------------------------------------------------------------------------
    # CÁLCULO DE LA NOTA FINAL (PROMEDIO NORMALIZADO)
    # --------------------------------------------------------------------------
    # Normalizamos todo de 0 a 1 para poder sumarlo
    def normalizar(series):
        return (series - series.min()) / (series.max() - series.min())

    score_corr = normalizar(corr_scores)
    score_mi   = normalizar(mi_scores)
    score_rf   = normalizar(rf_scores)
    
    # PROMEDIO: Damos el mismo peso a los 3 jueces (33% cada uno)
    nota_final = (score_corr + score_mi + score_rf) / 3
    
    # Ordenamos de mejor a peor
    ranking = nota_final.sort_values(ascending=False).head(30)
    
    # --------------------------------------------------------------------------
    # VISUALIZACIÓN
    # --------------------------------------------------------------------------
    print(f"\n🏆 TOP 30 SENSORES PARA {accion} (Ranking Combinado):")
    print("-" * 50)
    print(f"{'SENSOR':<15} | {'NOTA FINAL':<10} | {'Corr':<6} | {'MI':<6} | {'RF':<6}")
    print("-" * 50)
    
    for sensor, nota in ranking.items():
        # Recuperamos los valores originales normalizados para ver quién votó qué
        c = score_corr[sensor]
        m = score_mi[sensor]
        r = score_rf[sensor]
        # LOGICA DE COLORES
       # Función auxiliar para decidir el color de un valor concreto
        def obtener_color(valor):
            if valor >= 0.8:
                return C_GREEN    # >= 0.8: Verde
            elif valor >= 0.5:
                return C_RESET    # 0.5 a 0.79: Blanco (Normal)
            elif valor >= 0.3:
                return C_YELLOW   # 0.3 a 0.49: Amarillo
            else:
                return C_RED      # < 0.3: Rojo

        # Calculamos el color INDEPENDIENTE para cada columna
        col_n = obtener_color(nota)
        col_c = obtener_color(c)
        col_m = obtener_color(m)
        col_r = obtener_color(r)
            
        # Imprimimos: El nombre del sensor se queda en blanco, los números van con su color
        # Nota: Ponemos {C_RESET} después de cada número para que no pinte al siguiente
        print(f"{sensor:<15} | {col_n}{nota:.4f}{C_RESET}     | {col_c}{c:.2f}{C_RESET}   | {col_m}{m:.2f}{C_RESET}   | {col_r}{r:.2f}{C_RESET}")    
   
    # Generamos la lista lista para copiar
    print(f"\n📋 LISTA PARA COPIAR ({accion}):")
    lista_str = ", ".join([f"feature_idx['{s}']" for s in ranking.index])
    print(f"[{lista_str}]")

✅ Datos sincronizados: 125636 filas analizadas.

 🔍 ANALIZANDO: A_ACCEL
   1. Calculando Correlación Lineal...
   2. Calculando Información Mutua (Complejidad)...
   3. Entrenando Random Forest (Prueba de fuego)...

🏆 TOP 30 SENSORES PARA A_Accel (Ranking Combinado):
--------------------------------------------------
SENSOR          | NOTA FINAL | Corr   | MI     | RF    
--------------------------------------------------
S_track_9       | 0.8532     | 1.00   | 0.56   | 1.00
S_speed         | 0.7279     | 0.44   | 1.00   | 0.74
S_wheelSpinVel_0 | 0.5136     | 0.66   | 0.74   | 0.14
S_rpm           | 0.4486     | 0.78   | 0.54   | 0.02
S_track_8       | 0.4036     | 0.72   | 0.37   | 0.11
S_track_10      | 0.3956     | 0.75   | 0.34   | 0.10
S_track_4       | 0.2852     | 0.46   | 0.40   | 0.00
S_track_5       | 0.2405     | 0.43   | 0.28   | 0.01
S_track_14      | 0.2395     | 0.35   | 0.36   | 0.01
S_track_3       | 0.2392     | 0.37   | 0.34   | 0.01
S_track_11      | 0.2240     | 0.

In [3]:
import pandas as pd
import numpy as np

# Cargar datos
try:
    df = pd.read_csv('datos_limpios.csv')
    
    # Filtrar solo columnas de sensores (S_)
    cols_sensores = [c for c in df.columns if c.startswith('S_')]
    
    # Asegurar el orden correcto (segun csv header/training)
    # El orden en el CSV ya es el correcto normalmente, pero lo verificamos
    # X se crea como df[cols_sensores] en el notebook, asi que mantiene orden del CSV
    
    X = df[cols_sensores]
    
    # Calcular Mean y Std (Scale)
    scaler_mean = X.mean(axis=0).values
    scaler_scale = X.std(axis=0).values
    
    print("SCALER MEAN:")
    print(", ".join([f"{x:.8f}" for x in scaler_mean]))
    
    print("\nSCALER SCALE:")
    print(", ".join([f"{x:.8f}" for x in scaler_scale]))
    
    print(f"\nTotal columnas: {len(cols_sensores)}")
    print(f"Columnas: {cols_sensores}")

except Exception as e:
    print(f"Error: {e}")


SCALER MEAN:
-0.00205591, 2.64444126, 4415.52944382, 77.95180448, 7.61571627, 11.87461907, 19.39752355, 34.85956336, 41.98884369, 48.98707374, 54.87639929, 58.56179033, 48.84318519, 40.69321211, 33.78569296, 25.74149638, 17.39452623, 10.55262829, 7.62845053, 6.56981317, -0.04575499, 62.75078968, 0.34168918

SCALER SCALE:
0.06410456, 1.08946127, 1061.94762450, 36.68689551, 5.30684393, 10.65443270, 19.36858740, 43.89944727, 45.45835321, 44.62270662, 44.74142880, 49.63728173, 40.30549319, 35.36238996, 30.97158208, 23.30660485, 18.17308043, 10.47570956, 5.30996271, 2.97112274, 0.32603312, 32.22361920, 0.00356310

Total columnas: 23
Columnas: ['S_angle', 'S_gear', 'S_rpm', 'S_speed', 'S_track_0', 'S_track_3', 'S_track_4', 'S_track_5', 'S_track_6', 'S_track_7', 'S_track_8', 'S_track_9', 'S_track_10', 'S_track_11', 'S_track_12', 'S_track_13', 'S_track_14', 'S_track_15', 'S_track_16', 'S_track_17', 'S_trackPos', 'S_wheelSpinVel_0', 'S_z']
